In [70]:
import time

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [71]:
data = pd.read_json('./data/tcm_detail.json')
data

,title,href,medical_prescription,prescription_list,method_of_making,usage_and_dosage,excerpt
0,安肾丸,http://zhongyaofangji.com//a/anshenwan_12084.html,【处方】 肉桂 （去粗皮，不见火） 川乌 （炮，去皮、脐）各500克 桃仁 （麸炒） 白蒺...,"[肉桂, 川乌, 桃仁, 白蒺藜, 巴戟, 山药, 茯苓, 肉苁蓉, 石斛, 萆薢, 白术,...",【制法】上药为末，炼蜜为丸，如 梧桐子 大。,【用法用量】每服30丸，空腹时用温酒或 盐汤 送下； 小肠气 者，用炒 茴香 盐酒送下。,【摘录】 《太平惠民和剂局方》 卷五
1,百部丸,http://zhongyaofangji.com//b/baibuwan.html,【处方】 百部根 90克 升麻 15克 桂心 五味子 甘草 紫菀 干姜 ...,"[百部根, 升麻, 桂心, 五味子, 甘草, 紫菀, 干姜]",【制法】上为细末，炼蜜为丸， 梧桐子 大。,【用法用量】每服3克，食后、临卧前开水送下。,【摘录】 《备急千金要方》 卷十八
2,白矾丸,http://zhongyaofangji.com//b/baifanwan.html,【处方】 白矾 灰3钱，田父3分（烧灰）， 蛇蜕皮 1条（炒令微黄）， 青黛 1分（细研），...,"[白矾, 蛇蜕皮, 青黛, 鹤虱, 朱砂, 麝香, 芦荟, 莨菪子]",【制法】上为末，同研令匀，以烧饭为丸，如 绿豆 大。,【用法用量】每1岁儿服2丸，以粥饮送下。,【摘录】 《圣惠》 卷八十七
3,白茯苓汤,http://zhongyaofangji.com//b/baifulingtang.html,【处方】 白茯苓 （去黑皮）1两， 五灵脂 1两， 白芷 （微炒）1两， 黄明胶 （两片）1...,"[白茯苓, 五灵脂, 白芷, 黄明胶, 燥]",【制法】上为粗末。,【用法用量】每服3钱匕，水1盏，煎至8分，去滓，入蜜少许，更煎2沸，放温，细呷服，不拘时候。,【摘录】 《圣济总录》 卷八十八
4,百果酒,http://zhongyaofangji.com//b/baiguojiu.html,【处方】 香橼 2， 佛手 2个， 核桃 肉半斤， 龙眼肉 半斤， 莲肉 半斤， 橘饼 半斤...,"[香橼, 佛手, 核桃, 龙眼肉, 莲肉, 橘饼, 柏子仁, 松子, 红枣]",NaN,【用法用量】干 烧酒 50斤浸。,【摘录】 《仙拈集》 卷三
...,...,...,...,...,...,...,...
236,延寿丹,http://zhongyaofangji.com//y/yanshoudan.html,【处方】 天门冬 （去心） 远志 （去心） 山药 巴戟 各60克 赤石脂 车前子 ...,"[天门冬, 远志, 山药, 巴戟, 赤石脂, 车前子, 菖蒲, 柏子仁, 泽泻, 川椒, 熟...",【制法】上药为末，蜜丸 梧桐子 大。,【用法用量】每服70丸。,【摘录】 《丹溪心法》 卷三
237,玉真丸,http://zhongyaofangji.com//y/yuzhenwan.html,【处方】 龙骨 （捣罗，水飞三遍，重研令细，去水干，入熟绢袋盛，缝合，置米下，炊一遍了，干后...,"[龙骨, 菟丝子, 鹿茸, 韭子]",【制法】上四味，捣罗为末，炼 白蜜 和捣千下，丸如小豆大。,【用法用量】每次用温酒下7丸，一日二次。,【摘录】 《圣济总录》 卷一八五
238,钟乳泽兰圆,http://zhongyaofangji.com//z/zhongruzelanyuan....,【处方】 钟乳粉 三两， 泽兰 二两二钱半， 芜荑 （炒）半两， 麦门冬 （去心.焙）一两半...,"[钟乳粉, 泽兰, 芜荑, 麦门冬, 山茱萸, 艾叶, 防风, 柏子仁, 人参, 石膏, 石...",NaN,【用法用量】每服三十圆至五十圆，温酒或米饮下，空心，食前，日二服。,【摘录】《宋· 太平惠民和剂局方 》
239,驻世珍馐,http://zhongyaofangji.com//z/zhushizhenxiu.html,【处方】 当归 （酒洗）南芎 白芍 （酒炒） 熟地黄 菟丝子 （酒制） 巴戟 （酒浸，...,"[当归, 白芍, 熟地黄, 菟丝子, 巴戟, 肉苁蓉, 益智仁, 牛膝, 杜仲, 山药, 青...",【制法】上药共研细末，用豮 猪肉 不拘多少切片，酒炒熟，入药再炒，不可用水，瓷器牧贮。,【用法用量】空腹时好酒送下。,【摘录】 《万病回春》 卷四


In [72]:
data = data[['title', 'prescription_list']]
data

,title,prescription_list
0,安肾丸,"[肉桂, 川乌, 桃仁, 白蒺藜, 巴戟, 山药, 茯苓, 肉苁蓉, 石斛, 萆薢, 白术,..."
1,百部丸,"[百部根, 升麻, 桂心, 五味子, 甘草, 紫菀, 干姜]"
2,白矾丸,"[白矾, 蛇蜕皮, 青黛, 鹤虱, 朱砂, 麝香, 芦荟, 莨菪子]"
3,白茯苓汤,"[白茯苓, 五灵脂, 白芷, 黄明胶, 燥]"
4,百果酒,"[香橼, 佛手, 核桃, 龙眼肉, 莲肉, 橘饼, 柏子仁, 松子, 红枣]"
...,...,...
236,延寿丹,"[天门冬, 远志, 山药, 巴戟, 赤石脂, 车前子, 菖蒲, 柏子仁, 泽泻, 川椒, 熟..."
237,玉真丸,"[龙骨, 菟丝子, 鹿茸, 韭子]"
238,钟乳泽兰圆,"[钟乳粉, 泽兰, 芜荑, 麦门冬, 山茱萸, 艾叶, 防风, 柏子仁, 人参, 石膏, 石..."
239,驻世珍馐,"[当归, 白芍, 熟地黄, 菟丝子, 巴戟, 肉苁蓉, 益智仁, 牛膝, 杜仲, 山药, 青..."


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   title              241 non-null    object
 1   prescription_list  240 non-null    object
dtypes: object(2)
memory usage: 3.9+ KB


In [74]:
# 检查缺失值
nan_values = data.isna().sum()
nan_values

title                0
prescription_list    1
dtype: int64

In [75]:
data = data.dropna()
data

,title,prescription_list
0,安肾丸,"[肉桂, 川乌, 桃仁, 白蒺藜, 巴戟, 山药, 茯苓, 肉苁蓉, 石斛, 萆薢, 白术,..."
1,百部丸,"[百部根, 升麻, 桂心, 五味子, 甘草, 紫菀, 干姜]"
2,白矾丸,"[白矾, 蛇蜕皮, 青黛, 鹤虱, 朱砂, 麝香, 芦荟, 莨菪子]"
3,白茯苓汤,"[白茯苓, 五灵脂, 白芷, 黄明胶, 燥]"
4,百果酒,"[香橼, 佛手, 核桃, 龙眼肉, 莲肉, 橘饼, 柏子仁, 松子, 红枣]"
...,...,...
236,延寿丹,"[天门冬, 远志, 山药, 巴戟, 赤石脂, 车前子, 菖蒲, 柏子仁, 泽泻, 川椒, 熟..."
237,玉真丸,"[龙骨, 菟丝子, 鹿茸, 韭子]"
238,钟乳泽兰圆,"[钟乳粉, 泽兰, 芜荑, 麦门冬, 山茱萸, 艾叶, 防风, 柏子仁, 人参, 石膏, 石..."
239,驻世珍馐,"[当归, 白芍, 熟地黄, 菟丝子, 巴戟, 肉苁蓉, 益智仁, 牛膝, 杜仲, 山药, 青..."


In [76]:
# 展开列表
df_exploded = data['prescription_list'].explode()
df_exploded

0       肉桂
0       川乌
0       桃仁
0      白蒺藜
0       巴戟
      ... 
240     青皮
240     山药
240     茴香
240    山茱萸
240     木香
Name: prescription_list, Length: 2158, dtype: object

In [77]:
# 应用one-hot编码
df_dummies = pd.get_dummies(df_exploded)
df_dummies

,丁香,丹参,丹砂,乌头,乌梅肉,乌药,乳汁,乳香,五加皮,五味,...,黄连,黄酒,黄鳝,黑豆,鼠妇,鼠粘子,龙眼肉,龙骨,龙齿,龟版
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
240,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
240,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
240,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [78]:
# 聚合数据回到原始的DataFrame
df_one_hot = df_dummies.groupby(df_dummies.index).sum()
df_one_hot

,丁香,丹参,丹砂,乌头,乌梅肉,乌药,乳汁,乳香,五加皮,五味,...,黄连,黄酒,黄鳝,黑豆,鼠妇,鼠粘子,龙眼肉,龙骨,龙齿,龟版
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
# 对销量超过 1 的商品进行编码
def one_hot_encoder(k):
    if k <= 0:
        return 0
    if k >= 1:
        return 1
baskets_final = df_one_hot.iloc[:, 1:df_one_hot.shape[1]].applymap(one_hot_encoder)
# 找出最常一起出售的商品
frequent_itemsets = apriori(baskets_final, min_support=0.038, use_colnames=True, max_len=4).sort_values(by='support')
frequent_itemsets

D:\developing\anaconda3\envs\study_hei\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
157,0.041667,"(人参, 黄耆, 五味子)"
104,0.041667,"(当归, 牛膝)"
45,0.041667,(补骨脂)
77,0.041667,"(人参, 茯神)"
173,0.041667,"(川芎, 当归, 白术)"
...,...,...
48,0.200000,(附子)
28,0.220833,(白术)
12,0.225000,(当归)
25,0.316667,(甘草)


In [84]:
# 创建关联规则，指明前因后果项目
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values('lift', ascending=False)
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules

,antecedents,consequents,support,confidence,lift
27,(菟丝子),"(山茱萸, 肉苁蓉)",0.041667,0.416667,9.090909
26,"(山茱萸, 肉苁蓉)",(菟丝子),0.041667,0.909091,9.090909
24,"(菟丝子, 山茱萸)",(肉苁蓉),0.041667,0.909091,8.727273
29,(肉苁蓉),"(菟丝子, 山茱萸)",0.041667,0.400000,8.727273
178,"(菟丝子, 牛膝)",(肉苁蓉),0.045833,0.785714,7.542857
...,...,...,...,...,...
7,(牛膝),(当归),0.041667,0.256410,1.139601
35,(甘草),(木香),0.041667,0.131579,1.127820
34,(木香),(甘草),0.041667,0.357143,1.127820
230,(附子),(当归),0.050000,0.250000,1.111111


In [81]:
from pyecharts import options as opts
from pyecharts.charts import Graph
import time
nodes_data = set()

links = []
def map(data, MIN, MAX):
    """
    归一化映射到任意区间
    :param data: 数据
    :param MIN: 目标数据最小值
    :param MAX: 目标数据最小值
    :return:
    """
    d_min = data.min()  # 当前数据最大值
    d_max = data.max()  # 当前数据最小值
    return MIN + (MAX - MIN) / (d_max - d_min) * (data - d_min)


maplift = map(rules.lift, 0.1, 5)
maplift = maplift[::-1]
rules = rules[::-1]

for i in range(rules.shape[0]):
    item = rules.iloc[i]
    for antecedent in item.antecedents:
        for consequent in item.consequents:
            nodes_data.add(antecedent)
            nodes_data.add(consequent)
            links.append({
                "id": time.time(),
                "source": antecedent,
                "target": consequent,
                "lineStyle": opts.LineStyleOpts(width=maplift.iloc[i], color="source", curve=0.3),
            })

nodes = [{"name": x, "symbolSize": 5} for x in nodes_data]


categories = [{
    "name": "补虚中药关系"
}, ]
c = (
    Graph(init_opts=opts.InitOpts(width="1000px", height="1000px"))
    .add(
        "",
        nodes=nodes,
        links=links,
        categories=categories,
        layout="circular",
        is_rotate_label=True,
        itemstyle_opts=opts.ItemStyleOpts(color='#4B008299'),
        linestyle_opts=opts.LineStyleOpts(color="source", curve=0.3),
        label_opts=opts.LabelOpts(position="right"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Graph-Les Miserables"),
        legend_opts=opts.LegendOpts(orient="vertical", pos_left="2%", pos_top="20%"),
    )
    .render("graph_les_miserables_tcm.html")
)
